In [1]:
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as bs
import csv
import os
import sys
from requests import get

In [2]:
# Taking the URL into a variable
imdb_url = "http://www.imdb.com/search/title?release_date=2017,&page=1&ref_=adv_prv"

# Request URL Open and taking contents into a Variable
IMDB_data = uReq(imdb_url)

In [3]:
type(IMDB_data);

In [4]:
# Read The Data from HTTPResponse object

imdb_html = IMDB_data.read()

In [5]:
# Close the URL Open connection
IMDB_data.close()

In [6]:
# Displaying the contents after reading
type(imdb_html)
imdb_html;

In [7]:
# We need to parse the byte strem into proper html format using beautiful soup

page_soup = bs(imdb_html, 'html.parser')

In [8]:
type(page_soup);

In [9]:
# Displaying contents after properly formated into html using Beautiful soup
page_soup;
page_soup.prettify;

In [10]:
# Take all the bytes of stream into a containers
containers = page_soup.find_all('div', {'class' : "lister-item-content"})

In [11]:
type(containers)

bs4.element.ResultSet

In [12]:
# Checking the whole contents of container
containers;
len(containers)

50

In [13]:
# Checking the nth item set from resultset
nth_imdb = containers[6]

In [14]:
# Pull one-by-one record for nth Movie 
# Pickup the Movie Title
nth_imdb.a.text

'Pitch Perfect 3'

In [15]:
# Pickup Release date for 1st Movie
nth_imdb.h3.find('span', class_ ='lister-item-year').text

'(2017)'

In [16]:
# Pull the Movie runtime Duration
nth_imdb.find('span', {'class' : 'runtime'}).text

'93 min'

In [17]:
# pull the genres of the movie
nth_imdb.find('span', {'class' : 'genre'}).text.replace('\n', '').replace(',', '|').strip()

'Comedy| Music'

In [18]:
# Pull the rate of this Movie
nth_imdb.strong.text

'6.3'

In [19]:
# Pull the MetaScore of the Movie
nth_imdb.find('span', class_ ='metascore').text.strip()

'40'

In [20]:
# Pull the No of votes of this movie
nth_imdb.find('span', {'name' : 'nv'}).text.replace(',', '')

'8623'

In [21]:
# we put the condition, if metascore value none then skip that movie name
# Print the result set

for item in containers:
    if item.find('span', class_ ='metascore') is not None:
        Title = item.h3.a.text.replace(',','')
        Release_Yrs = item.find('span', class_ ='lister-item-year text-muted unbold').text.replace('(','').replace(')','')
        Runtime = item.find('span', {'class' : 'runtime'}).text
        Genres = item.find('span', {'class' : 'genre'}).text.replace('\n', '').replace(',', '|').strip()
        Rating = float(item.strong.text)
        MetaScore = int(item.find('span', class_ ='metascore').text.strip())
        Votes = int(item.find('span', {'name' : 'nv'}).text.replace(',', ''))
        print(Title, Release_Yrs,Runtime, Genres,Rating, MetaScore, Votes)

Jumanji: Welcome to the Jungle 2017 119 min Action| Adventure| Comedy 7.2 58 43629
Star Wars: Episode VIII - The Last Jedi 2017 152 min Action| Adventure| Fantasy 7.5 85 279576
The Greatest Showman 2017 105 min Biography| Drama| Musical 8.0 48 37532
Bright I 2017 117 min Action| Crime| Fantasy 6.5 29 87868
Blade Runner 2049 2017 164 min Drama| Mystery| Sci-Fi 8.3 81 200660
The Shape of Water 2017 123 min Adventure| Drama| Fantasy 8.1 86 18618
Pitch Perfect 3 2017 93 min Comedy| Music 6.3 40 8623
Justice League 2017 120 min Action| Adventure| Fantasy 7.0 45 168710
Molly's Game 2017 140 min Biography| Drama 7.7 71 10755
Dunkirk 2017 106 min Action| Drama| History 8.1 94 311113
Coco I 2017 105 min Animation| Adventure| Comedy 8.7 81 57647
Thor: Ragnarok 2017 130 min Action| Adventure| Comedy 8.1 74 184040
It I 2017 135 min Drama| Horror| Thriller 7.6 69 221144
The Disaster Artist 2017 104 min Biography| Comedy| Drama 7.9 76 28999
Three Billboards Outside Ebbing Missouri 2017 115 min Crime

#### Write the result set into CSV file

In [38]:
# Write the result set into a csv file
Titles = []
ReleaseYears = []
Runtime = []
Genres = []
Ratings = []
Metascores = []
Votes = []
filename = "C:\DataAnalysis\Data\webscrap\MovieList.csv"
f= open(filename,'w', encoding="utf-8")
header = 'Title, Release_Yrs, Runtime, Genres, Rating, MetaScore,Votes \n'
f.write(header)
for items in containers:
    if items.find('span', class_ ='metascore') is not None:
        Title = items.h3.a.text.replace(',','')
        Titles.append(Title)
        
        Release_Year = items.find('span', class_ ='lister-item-year').text.replace('(','').replace(')','')
        ReleaseYears.append(Release_Year)
        
        Duration = items.find('span', {'class' : 'runtime'}).text
        Runtime.append(Duration)
        
        Genre = items.find('span', {'class' : 'genre'}).text.replace('\n', '').replace(',', '|').strip()
        Genres.append(Genre)
        
        Rating = items.strong.text
        Ratings.append(float(Rating))
        
        MetaScore = items.find('span', class_ ='metascore').text.strip()
        Metascores.append(MetaScore)
        
        Vote = items.find('span', {'name' : 'nv'}).text.replace(',', '')
        Votes.append(Vote)
        f.write(Title + ", " + Release_Year + ", " + Duration + ", " + Genre + ", " + Rating + ", " + MetaScore + "," + Vote + "\n")
        #print(Titles + ", " + ReleaseYears + ", " + Runtime + ", " + Genres + ", " + Ratings + ", " + Metascores + "," + Votes +)
f.close()


#### Let's display the data into tabular format

In [39]:
type(Titles)

list

In [40]:
import pandas as pd
import numpy as np

In [41]:
IMDB_DF = pd.DataFrame({'Title' : Titles,
                       'Year' : ReleaseYears,
                       'Runtime': Runtime,
                       'Genres' : Genres,
                       'Rating': Ratings,
                       'Metascore' : Metascores,
                       'Votes' : Votes
                       })


In [31]:
IMDB_DF.head()

,Genres,Metascore,Rating,Runtime,Title,Votes,Year
0,Action| Adventure| Comedy,58,7.2,119 min,Jumanji: Welcome to the Jungle,43629,2017
1,Action| Adventure| Fantasy,85,7.5,152 min,Star Wars: Episode VIII - The Last Jedi,279576,2017
2,Biography| Drama| Musical,48,8.0,105 min,The Greatest Showman,37532,2017
3,Action| Crime| Fantasy,29,6.5,117 min,Bright,87868,I 2017
4,Drama| Mystery| Sci-Fi,81,8.3,164 min,Blade Runner 2049,200660,2017
